In [6]:
import xarray as xr

ds = xr.open_dataset(r"N:\LUF-Modelling\LUTO2_XH\LUTO2\output\20250912_Price_Task_RES13_year5\Run_1_GHG_high_BIO_high_CUT_50\output\2025_09_12__16_06_16_RF13_2010-2050\out_2020\xr_biodiversity_GBF2_priority_ag_2020.nc")
print(ds)

<xarray.Dataset> Size: 8MB
Dimensions:  (cell: 24987, lu: 28, lm: 3)
Coordinates:
  * cell     (cell) int32 100kB 0 1 2 3 4 5 ... 24982 24983 24984 24985 24986
  * lu       (lu) <U27 3kB 'Apples' 'Beef - modified land' ... 'Winter oilseeds'
  * lm       (lm) <U3 36B 'ALL' 'dry' 'irr'
Data variables:
    data     (cell, lu, lm) float32 8MB ...


In [7]:
ds = xr.open_dataset(r"N:\LUF-Modelling\LUTO2_XH\LUTO2\output\20250912_Price_Task_RES13_year5\Run_1_GHG_high_BIO_high_CUT_50\output\2025_09_12__16_06_16_RF13_2010-2050\out_2020\xr_GHG_ag_2020.nc")
print(ds)

<xarray.Dataset> Size: 118MB
Dimensions:     (lu: 28, cell: 24987, lm: 3, GHG_source: 14)
Coordinates:
  * lu          (lu) <U27 3kB 'Apples' ... 'Winter oilseeds'
  * cell        (cell) int64 200kB 0 1 2 3 4 5 ... 24982 24983 24984 24985 24986
  * lm          (lm) <U3 36B 'ALL' 'dry' 'irr'
  * GHG_source  (GHG_source) <U53 3kB 'ALL' 'Chemical Application' ... 'Sowing'
Data variables:
    data        (cell, GHG_source, lm, lu) float32 118MB ...
